In [1]:
import pandas as pd
import glob
import os
import matplotlib.pyplot as plt
import numpy as np
from scipy.signal import find_peaks
from scipy import signal
import statistics as st
from ipynb.fs.full.PreProcessing import Filter,plot,integrate
from ipynb.fs.full.GaitEventDetection import MidStance,heelStrike,toeOff
from ipynb.fs.full.spatiotemporal import functionstridetime,TSwOnset, functionswingtime,functionstancetime

C:\Users\User\Anaconda3\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
def read_csv_file(file_path):
    try:
        return pd.read_csv(file_path, encoding='utf-8', sep=',')
    except UnicodeDecodeError:
        return pd.read_csv(file_path, encoding='latin1', sep=',')

In [3]:
womac_path = "csv/final_womac.csv"

wdf = read_csv_file(womac_path)
wdf

,name,gender,leg,age,weight,height,pain,stifness,phy_func
0,marwa abdelaly,f,right,40.0,95.0,163,7,2,24
1,amal kamel,f,left,55.0,76.0,150,19,8,66
2,sanaa abd-elhusin,f,right,49.0,85.0,150,9,4,41
3,mohamed saad,m,right,40.0,87.0,176,0,0,13
4,fayaa ahmed,f,right,36.0,70.0,156,14,4,44
5,aliaa ateya,f,right,34.0,65.0,153,14,7,43
6,zakareia reda,m,right,58.0,103.0,175,5,0,7
7,sameh mostafa,m,right,43.0,138.0,183,0,3,4
8,samia abd-allah,f,left,NaN,65.0,152,10,16,48
9,rehab helmy,m,right,35.0,70.0,155,8,0,14


In [5]:
# BMI
wdf['height_m'] = wdf['height'] / 100
wdf['BMI'] = wdf['weight'] / (wdf['height_m'] ** 2)
wdf.drop(columns=['height_m'], inplace=True)

# Total score
required_columns = ['pain', 'stifness', 'phy_func']
wdf['total_score'] = wdf[required_columns].sum(axis=1)
   
# Severity score
# wdf["severity_score"] = wdf['total_score'].apply(lambda x: x * 1.04)

# Severity
# WOMAC scores below 36, between 36 and 60, and above 60 were categorized as mild, moderate, and severe, 
# bins = [0, 24, 48, 72, 100]
# labels = ['normal', 'mild', 'moderate', 'severe']
bins = [0, 24, 36, 60, 96]
labels = ['normal', 'mild',  'moderate', 'severe']
# bins = [0,36, 96]
# labels = ['no',  'yes']
wdf['severity'] = pd.cut(wdf['total_score'], bins=bins, labels=labels, right=True, include_lowest=True)

wdf['patient_id'] = range(1, len(wdf) + 1)

columns_to_drop = ['weight', 'height', 'pain', 'stifness', 'phy_func', 'total_score']
wdf = wdf.drop(columns=columns_to_drop)

wdf

,name,gender,leg,age,BMI,severity,patient_id
0,marwa abdelaly,f,right,40.0,35.755956,mild,1
1,amal kamel,f,left,55.0,33.777778,severe,2
2,sanaa abd-elhusin,f,right,49.0,37.777778,moderate,3
3,mohamed saad,m,right,40.0,28.086260,normal,4
4,fayaa ahmed,f,right,36.0,28.763971,severe,5
5,aliaa ateya,f,right,34.0,27.767098,severe,6
6,zakareia reda,m,right,58.0,33.632653,normal,7
7,sameh mostafa,m,right,43.0,41.207561,normal,8
8,samia abd-allah,f,left,NaN,28.133657,severe,9
9,rehab helmy,m,right,35.0,29.136316,normal,10


In [6]:
wdf["severity"].value_counts().to_dict()

{'normal': 9, 'severe': 6, 'mild': 4, 'moderate': 4}

In [ ]:
# # BMI
# wdf['height_m'] = wdf['height'] / 100
# wdf['BMI'] = wdf['weight'] / (wdf['height_m'] ** 2)
# wdf.drop(columns=['height_m'], inplace=True)

# # Total score
# required_columns = ['pain', 'stifness', 'phy_func']
# wdf['total_score'] = wdf[required_columns].sum(axis=1)
   
# # Severity score
# # wdf["severity_score"] = wdf['total_score'].apply(lambda x: x * 1.04)

# # Severity
# # WOMAC scores below 36, between 36 and 60, and above 60 were categorized as mild, moderate, and severe, 
# # bins = [0, 24, 48, 72, 100]
# # labels = ['normal', 'mild', 'moderate', 'severe']
# bins = [0, 36, 60, 100]
# labels = ['mild',  'moderate', 'severe']
# wdf['severity'] = pd.cut(wdf['total_score'], bins=bins, labels=labels, right=True, include_lowest=True)

# wdf['patient_id'] = range(1, len(wdf) + 1)

# columns_to_drop = ['weight', 'height', 'pain', 'stifness', 'phy_func', 'total_score']
# wdf = wdf.drop(columns=columns_to_drop)

# wdf
# wdf["severity"].value_counts().to_dict()

In [8]:
wdf.to_csv("csv/data_training/classify.csv", index=False)